In [168]:
from datetime import datetime, timedelta
from collections import defaultdict

# === INPUTS ===

user_preferences = {
    "optimalFocusDuration": "30–45 minutes",
    "breakDuration": 15,
    "userStudyStyle": "multiple_passes",  # "multiple_passes" or "one_pass_deep"
    "preferStudyTime": ["late morning", "afternoon", "night"]
}

availability = {
    "12/02/2025": ["09:00-11:00", "14:00-16:00", "20:00-22:00"],
    "14/02/2025": ["10:00-12:00", "14:00-18:00"],
    "15/02/2025": ["08:00-11:00", "14:00-16:00"],
    "17/02/2025": ["10:00-12:00", "14:00-17:00"]
}

courses = [
    {
        "name": "Information Retrieval",
        "topics": [
            {"name": "Vector Space Model", "difficulty": 3, "understanding": 5, "studyTime": 60},
            {"name": "BM25", "difficulty": 4, "understanding": 2, "studyTime": 90}
        ]
    },
    {
        "name": "Software Architecture",
        "topics": [
            {"name": "Introduction", "difficulty": 1, "understanding": 5, "studyTime": 30},
            {"name": "System Architecture", "difficulty": 4, "understanding": 1, "studyTime": 120}
        ]
    }
]

# === HELPERS ===

def get_focus_minutes(pref):
    mapping = {
        "~30 minutes": 30,
        "30–45 minutes": 45,
        "1–2 hours": 90,
        "Over 2 hours": 120
    }
    return mapping.get(pref, 30)

def estimate_sessions(difficulty, understanding, study_time, study_style):
    """Estimate sessions based on study style, difficulty, and understanding."""
    adjustment = (difficulty - understanding) * 0.1
    adjusted_study_time = study_time * (1 + adjustment)

    if study_style == "multiple_passes":
        overview_time = adjusted_study_time * 0.2
        deep_time = adjusted_study_time * 0.6
        review_time = adjusted_study_time * 0.2
        return [overview_time, deep_time, review_time]
    else:  # one_pass_deep
        return [adjusted_study_time]

def minutes_between(start_str, end_str):
    fmt = "%d/%m/%Y %H:%M"
    return int((datetime.strptime(end_str, fmt) - datetime.strptime(start_str, fmt)).total_seconds() // 60)

# === RULE-BASED SYSTEM ===

def apply_rule_based_adjustments(courses, user_preferences):
    adjusted_sessions = []
    focus_minutes = get_focus_minutes(user_preferences["optimalFocusDuration"])

    for course in courses:
        for topic in course["topics"]:
            difficulty = topic["difficulty"]
            understanding = topic["understanding"]
            base_time = topic["studyTime"]

            # Rule 1: More sessions if difficulty gap is high
            if difficulty - understanding >= 2:
                number_of_sessions = 3
            elif difficulty >= 3:
                number_of_sessions = 2
            else:
                number_of_sessions = 1

            # Rule 2: Study style (multiple passes), only if base_time is big enough
            if user_preferences["userStudyStyle"] == "multiple_passes" and base_time >= focus_minutes:
                session_length = min(base_time / number_of_sessions, focus_minutes)
                overview = session_length * 0.2
                deep = session_length * 0.6
                review = session_length * 0.2
                session_durations = [overview, deep, review]
            else:
                # If study time is small, just make 1 session
                session_length = min(base_time, focus_minutes)
                session_durations = [session_length]

            # Create sessions
            for idx, duration in enumerate(session_durations, 1):
                session_name = f"{course['name']} - {topic['name']} - S{idx}"
                adjusted_sessions.append((session_name, duration, difficulty))
    
    return adjusted_sessions

# === TIME SLOT GENERATION ===

def generate_time_slots(availability, focus_minutes, break_minutes):
    slots = []
    for day, blocks in availability.items():
        for block in blocks:
            start_str, end_str = block.split("-")
            t_start = datetime.strptime(f"{day} {start_str}", "%d/%m/%Y %H:%M")
            t_end = datetime.strptime(f"{day} {end_str}", "%d/%m/%Y %H:%M")

            while t_start + timedelta(minutes=focus_minutes) <= t_end:
                slot_start = t_start.strftime("%d/%m/%Y %H:%M")
                slot_end = (t_start + timedelta(minutes=focus_minutes)).strftime("%d/%m/%Y %H:%M")
                slots.append((slot_start, slot_end))
                t_start += timedelta(minutes=focus_minutes + break_minutes)
    return slots

def is_within_preferred(slot_start, prefer_study_times):
    time_periods = {
        "early morning": (4, 8),
        "late morning": (8, 12),
        "afternoon": (12, 18),
        "evening": (18, 22),
        "night": (22, 24),
        "late night": (0, 4)
    }
    slot_dt = datetime.strptime(slot_start, "%d/%m/%Y %H:%M")
    hour = slot_dt.hour

    for period in prefer_study_times:
        start_hour, end_hour = time_periods.get(period, (0, 24))
        if start_hour <= hour < end_hour or (start_hour > end_hour and (hour >= start_hour or hour < end_hour)):
            return True
    return False

# === CSP ASSIGNMENT ===

occupied_slots = set()

def assign_time_slot(session_name, required_minutes, difficulty, domain, user_preferences):
    preferred_slots = []
    normal_slots = []

    for slot_start, slot_end in domain:
        if (slot_start, slot_end) not in occupied_slots:
            available_minutes = minutes_between(slot_start, slot_end)
            if available_minutes >= required_minutes:
                if difficulty >= 3 and is_within_preferred(slot_start, user_preferences["preferStudyTime"]):
                    preferred_slots.append((slot_start, slot_end))
                else:
                    normal_slots.append((slot_start, slot_end))

    if preferred_slots:
        selected = preferred_slots[0]
    elif normal_slots:
        selected = normal_slots[0]
    else:
        return None

    occupied_slots.add(selected)
    return selected

def prepare_sessions(courses, user_preferences):
    """Prepare sessions grouped by course, sorted by topic difficulty."""
    sessions_by_course = defaultdict(list)

    for course in courses:
        # Sort topics inside the course by difficulty (easy ➔ hard)
        sorted_topics = sorted(course["topics"], key=lambda x: x["difficulty"])
        for topic in sorted_topics:
            # Estimate sessions for each topic
            session_times = estimate_sessions(topic["difficulty"], topic["understanding"], topic["studyTime"], user_preferences["userStudyStyle"])
            for idx, session_time in enumerate(session_times, 1):
                session_name = f"{course['name']} - {topic['name']} - S{idx}"
                sessions_by_course[course["name"]].append((session_name, session_time, topic["difficulty"]))  # We keep difficulty too
    return sessions_by_course

In [169]:
if __name__ == "__main__":
    # Phase 1: Apply Rule-Based adjustments
    adjusted_sessions = apply_rule_based_adjustments(courses, user_preferences)

    # Phase 2: CSP setup
    focus_minutes = get_focus_minutes(user_preferences["optimalFocusDuration"])
    domain = generate_time_slots(availability, focus_minutes, user_preferences["breakDuration"])

    study_plan = {}

    # Phase 3: Assign sessions
    for session_name, required_minutes, difficulty in adjusted_sessions:
        slot = assign_time_slot(session_name, required_minutes, difficulty, domain, user_preferences)
        if slot:
            study_plan[session_name] = slot
        else:
            print(f"No valid time slot available for {session_name}")

    # Phase 4: Organize and print study plan
    if study_plan:
        organized_plan = defaultdict(list)
        for session, (start_time, end_time) in study_plan.items():
            date = start_time.split(" ")[0]
            organized_plan[date].append((start_time, end_time, session))

        print("Study Plan:")
        for date in sorted(organized_plan.keys()):
            print(f"\n{date}")
            for start, end, session in sorted(organized_plan[date]):
                print(f"  {start.split(' ')[1]} - {end.split(' ')[1]} ➔ {session}")
    else:
        print("No study plan generated.")

Study Plan:

12/02/2025
  09:00 - 09:45 ➔ Information Retrieval - Vector Space Model - S1
  10:00 - 10:45 ➔ Information Retrieval - Vector Space Model - S2
  14:00 - 14:45 ➔ Information Retrieval - Vector Space Model - S3
  15:00 - 15:45 ➔ Information Retrieval - BM25 - S1
  20:00 - 20:45 ➔ Software Architecture - Introduction - S1

14/02/2025
  10:00 - 10:45 ➔ Information Retrieval - BM25 - S2
  11:00 - 11:45 ➔ Information Retrieval - BM25 - S3
  14:00 - 14:45 ➔ Software Architecture - System Architecture - S1
  15:00 - 15:45 ➔ Software Architecture - System Architecture - S2
  16:00 - 16:45 ➔ Software Architecture - System Architecture - S3


In [170]:
from datetime import datetime, timedelta
from collections import defaultdict

## Inputs

In [171]:
# === INPUTS ===

user_preferences = {
  "preferredStudyTimes": ["late morning", "afternoon", "night"],
  "preferredSessionDuration": {
    "min": 30,
    "max": 90
  },
  "revisionFrequency": "2-3 reviews per topic",
  "breakDuration": 15
}

availability = {
    "12/02/2025": ["09:00-11:00", "14:00-16:00", "20:00-22:00"],
    "14/02/2025": ["10:00-12:00", "14:00-18:00", "20:00-22:00"],
    "15/02/2025": ["08:00-11:00", "14:00-16:00", "20:00-22:00"],
    "17/02/2025": ["10:00-12:00", "14:00-17:00", "20:00-22:00"]
}

courses = [
    {
        "name": "Operating Systems",
        "credit": 3,
        "examDate": "25/02/2025",
        "examTime": "09:00–11:00",
        "topics": [
            {"title": "CPU Scheduling", "difficulty": 3, "confidence": 2, "studyTime": 90},
            {"title": "Deadlocks", "difficulty": 4, "confidence": 3, "studyTime": 75},
            {"title": "Virtual Memory", "difficulty": 5, "confidence": 2, "studyTime": 120}
        ]
    },
    {
        "name": "Data Structures and Algorithms",
        "credit": 3,
        "examDate": "22/02/2025",
        "examTime": "10:00–12:00",
        "topics": [
            {"title": "Trees & Graphs", "difficulty": 3, "confidence": 2, "studyTime": 80},
            {"title": "Sorting Algorithms", "difficulty": 2, "confidence": 4, "studyTime": 50},
            {"title": "Dynamic Programming", "difficulty": 5, "confidence": 2, "studyTime": 110}
        ]
    },
    {
        "name": "Machine Learning",
        "credit": 3,
        "examDate": "23/02/2025",
        "examTime": "13:00–15:00",
        "topics": [
            {"title": "Regression Models", "difficulty": 3, "confidence": 3, "studyTime": 70},
            {"title": "Classification", "difficulty": 4, "confidence": 2, "studyTime": 90},
            {"title": "Neural Networks", "difficulty": 5, "confidence": 2, "studyTime": 130}
        ]
    },
    {
        "name": "Database Systems",
        "credit": 2,
        "examDate": "27/02/2025",
        "examTime": "09:30–11:30",
        "topics": [
            {"title": "SQL Joins", "difficulty": 2, "confidence": 4, "studyTime": 40},
            {"title": "Normalization", "difficulty": 3, "confidence": 3, "studyTime": 60},
            {"title": "Indexing & Query Optimization", "difficulty": 4, "confidence": 2, "studyTime": 90}
        ]
    },
    {
        "name": "Computer Networks",
        "credit": 2,
        "examDate": "25/02/2025",
        "examTime": "14:00–16:00",
        "topics": [
            {"title": "TCP/IP Model", "difficulty": 2, "confidence": 4, "studyTime": 50},
            {"title": "Routing Protocols", "difficulty": 4, "confidence": 2, "studyTime": 100},
            {"title": "Congestion Control", "difficulty": 3, "confidence": 3, "studyTime": 60}
        ]
    }
]

assignments = [
    {
        "course": "Computer Networks",
        "title": "Midterm Essay",
        "associatedTopic": ["TCP/IP Model"],
        "dueDate": "14/02/2025",
        "time": "15.00",
        "estimatedTime": 50 
    },
    {
        "course": "Machine Learning",
        "title": "Final Project",
        "assiociatedTopic": ["Regression Models", "Classification", "Neural Networks"],
        "dueDate": "10/03/2025",
        "estimatedTime": 180
    }
]

## Helpers

In [172]:
def get_preferred_time_periods(pref):
    time_periods = {
            "early morning": (4, 8),
            "late morning": (8, 12),
            "afternoon": (12, 18),
            "evening": (18, 22),
            "night": (22, 24),
            "late night": (0, 4)
        }
    return [time_periods[p] for p in pref if p in time_periods]

In [173]:
preferred_ranges = get_preferred_time_periods(user_preferences["preferredStudyTimes"])
print(preferred_ranges)

[(8, 12), (12, 18), (22, 24)]


In [174]:
def round10(x):
        return round(x / 10) * 10

In [175]:
def estimate_study_time(difficulty, confidence, base_time=None):
    """
    Estimate adjusted study time based on difficulty and confidencial.
    Rounds the result to the nearest 10 minutes.
    """
    if base_time is None:
        base_time = 60  # default if not provided

    adjustment = (difficulty - confidence) * 0.1
    adjusted_time = base_time * (1 + adjustment)

    # Clamp to minimum of 15 minutes, then round to nearest 10
    adjusted_time = round10(max(15, adjusted_time))

    return adjusted_time

In [176]:
study_time = estimate_study_time(1, 5, 30)
print(study_time)

20


In [177]:
import math

def estimate_topic_sessions(adjusted_study_time, revision_freq, preferred_study_duration):
    """
    Break adjusted study time into sessions based on revision strategy.
    Enforce preferred session duration by splitting oversized chunks.
    All durations are rounded UP to the nearest 10 minutes.
    """

    min_dur = preferred_study_duration["min"]
    max_dur = preferred_study_duration["max"]

    sessions = []

    if revision_freq == "single deep review before exam":
        deep = round10(adjusted_study_time)
        review = round10(adjusted_study_time * 0.2)
        sessions = [deep, review]

    elif revision_freq == "2-3 reviews per topic":
        overview = round10(adjusted_study_time * 0.2)
        deep = round10(adjusted_study_time)
        review = round10(adjusted_study_time * 0.3)
        sessions = [overview, deep, review]

    elif revision_freq == "daily review sessions":
        review = round10(adjusted_study_time * 0.25)
        core_time = adjusted_study_time

        # 🗓️ Target up to 5 daily sessions (or as many fit given min_dur)
        num_daily = min(5, max(1, core_time // min_dur))

        base = core_time // num_daily
        remainder = core_time % num_daily

        sessions = []
        for i in range(num_daily):
            session_length = base + (1 if i < remainder else 0)
            sessions.append(round10(session_length))

        sessions.append(review)

    else:
        core = round10(adjusted_study_time)
        review = round10(adjusted_study_time * 0.2)
        sessions = [core, review]

    # 💡 Enforce preferred max duration (split if too long)
    final_sessions = []
    for s in sessions:
        if s > max_dur:
            parts = math.ceil(s / max_dur)
            per_part = round10(s / parts)
            final_sessions.extend([per_part] * parts)
        else:
            final_sessions.append(s)

    return final_sessions

def create_assignment_sessions(estimated_time, preferred_study_duration):
    # min_dur from preferred_study_duration is NOT used for assignments.
    max_dur = preferred_study_duration["max"]

    if estimated_time <= 0: # Handles 0 and any potential negative input
        return []

    # Case 1: Estimated time is small enough for a single session (<= max_dur)
    if estimated_time <= max_dur:
        session_duration = round10(estimated_time)
        # round10(positive_small_number) will be at least 10.
        # We expect estimated_time > 0 here due to the check above.
        return [session_duration]

    # Case 2: Estimated time is greater than max_dur, requires splitting
    else:
        num_parts = math.ceil(estimated_time / float(max_dur)) # Number of sessions
        
        # Calculate what each part would be if total time was distributed, then round it.
        # This initial rounding might make the sum of parts larger than estimated_time.
        per_part_ideal_rounded = round10(estimated_time / num_parts)
        
        sessions = [per_part_ideal_rounded] * int(num_parts)

        # Adjust sum if rounding per_part_ideal_rounded up made the total too high
        current_sum = sum(sessions)
        if current_sum > estimated_time and len(sessions) > 0:
            difference_to_cut = current_sum - estimated_time
            
            # Value of the last session before its own final rounding
            last_session_unrounded_adjusted_value = sessions[-1] - difference_to_cut
            
            if last_session_unrounded_adjusted_value > 0:
                sessions[-1] = round10(last_session_unrounded_adjusted_value)
            else:
                # If reducing the last session makes its unrounded value non-positive,
                # it means the other sessions (due to their own rounding up)
                # already cover or exceed the estimated_time.
                # Mark this last session as 0 to be filtered out.
                sessions[-1] = 0
        
        # Filter out any zero-duration sessions that might have resulted from the adjustment
        final_sessions = [s_duration for s_duration in sessions if s_duration > 0]
        
        return final_sessions


In [178]:
estimate_topic_sessions(20, "2-3 reviews per topic", user_preferences["preferredSessionDuration"])

[0, 20, 10]

## Result

In [ ]:
print("--- Topic Sessions ---")
for course in courses:
    print(f"\n📘 Course: {course['name']}")
    for topic in course["topics"]:
        base_time = topic.get("studyTime", 60) # Default to 60 if studyTime not present
        adjusted_time = estimate_study_time(topic["difficulty"], topic["confidence"], base_time)
        # Using the 'estimate_topic_sessions' function as it's designed for topics
        sessions = estimate_topic_sessions(adjusted_time, user_preferences["revisionFrequency"], user_preferences["preferredSessionDuration"])
        session_info = ", ".join([f"{int(s)} min" for s in sessions])
        print(f"  • {topic['title']} → Total Adjusted: {adjusted_time} min → Sessions: {session_info}")

print("\n\n--- Assignment Sessions ---") # Added extra newline for separation
for assignment in assignments:
    print(f"\n🛠️ Assignment: {assignment['title']} (Course: {assignment.get('course', 'N/A')})")
    estimated_time = assignment["estimatedTime"]
    assignment_sessions_durations = create_assignment_sessions(estimated_time, user_preferences["preferredSessionDuration"])
    session_info = ", ".join([f"{int(s)} min" for s in assignment_sessions_durations])
    print(f"  • Total Estimated: {estimated_time} min → Sessions: {session_info}")

--- Topic Sessions ---

📘 Course: Operating Systems
  • CPU Scheduling → Total Adjusted: 100 min → Sessions: 20 min, 50 min, 50 min, 30 min
  • Deadlocks → Total Adjusted: 80 min → Sessions: 20 min, 80 min, 20 min
  • Virtual Memory → Total Adjusted: 160 min → Sessions: 30 min, 80 min, 80 min, 50 min

📘 Course: Data Structures and Algorithms
  • Trees & Graphs → Total Adjusted: 90 min → Sessions: 20 min, 90 min, 30 min
  • Sorting Algorithms → Total Adjusted: 40 min → Sessions: 10 min, 40 min, 10 min
  • Dynamic Programming → Total Adjusted: 140 min → Sessions: 30 min, 70 min, 70 min, 40 min

📘 Course: Machine Learning
  • Regression Models → Total Adjusted: 70 min → Sessions: 10 min, 70 min, 20 min
  • Classification → Total Adjusted: 110 min → Sessions: 20 min, 60 min, 60 min, 30 min
  • Neural Networks → Total Adjusted: 170 min → Sessions: 30 min, 80 min, 80 min, 50 min

📘 Course: Database Systems
  • SQL Joins → Total Adjusted: 30 min → Sessions: 10 min, 30 min, 10 min
  • Normaliz

In [180]:
import math
from datetime import datetime, timedelta

# --- DATA (from previous inputs) ---
user_preferences = {
  "preferredStudyTimes": ["late morning", "afternoon", "night"], # late morning (8-12), afternoon (12-18), night (22-24)
  "preferredSessionDuration": { "min": 30, "max": 90 },
  "revisionFrequency": "2-3 reviews per topic",
  "breakDuration": 15
}

availability_raw = {
    "12/02/2025": ["09:00-11:00", "14:00-16:00", "20:00-22:00"],
    "14/02/2025": ["10:00-12:00", "14:00-18:00", "20:00-22:00"],
    "15/02/2025": ["08:00-11:00", "14:00-16:00", "20:00-22:00"],
    "17/02/2025": ["10:00-12:00", "14:00-17:00", "20:00-22:00"]
}

courses_raw = [
    {
        "name": "Operating Systems", "credit": 3, "examDate": "25/02/2025", "examTime": "09:00–11:00",
        "topics": [
            {"title": "CPU Scheduling", "difficulty": 3, "confidence": 2, "studyTime": 90},
            {"title": "Deadlocks", "difficulty": 4, "confidence": 3, "studyTime": 75},
            {"title": "Virtual Memory", "difficulty": 5, "confidence": 2, "studyTime": 120}
        ]
    },
    {
        "name": "Data Structures and Algorithms", "credit": 3, "examDate": "22/02/2025", "examTime": "10:00–12:00",
        "topics": [
            {"title": "Trees & Graphs", "difficulty": 3, "confidence": 2, "studyTime": 80},
            {"title": "Sorting Algorithms", "difficulty": 2, "confidence": 4, "studyTime": 50},
            {"title": "Dynamic Programming", "difficulty": 5, "confidence": 2, "studyTime": 110}
        ]
    },
    {
        "name": "Machine Learning", "credit": 3, "examDate": "23/02/2025", "examTime": "13:00–15:00",
        "topics": [
            {"title": "Regression Models", "difficulty": 3, "confidence": 3, "studyTime": 70},
            {"title": "Classification", "difficulty": 4, "confidence": 2, "studyTime": 90},
            {"title": "Neural Networks", "difficulty": 5, "confidence": 2, "studyTime": 130}
        ]
    },
    {
        "name": "Database Systems", "credit": 2, "examDate": "27/02/2025", "examTime": "09:30–11:30",
        "topics": [
            {"title": "SQL Joins", "difficulty": 2, "confidence": 4, "studyTime": 40},
            {"title": "Normalization", "difficulty": 3, "confidence": 3, "studyTime": 60},
            {"title": "Indexing & Query Optimization", "difficulty": 4, "confidence": 2, "studyTime": 90}
        ]
    },
    {
        "name": "Computer Networks", "credit": 2, "examDate": "25/02/2025", "examTime": "14:00–16:00",
        "topics": [
            {"title": "TCP/IP Model", "difficulty": 2, "confidence": 4, "studyTime": 50},
            {"title": "Routing Protocols", "difficulty": 4, "confidence": 2, "studyTime": 100},
            {"title": "Congestion Control", "difficulty": 3, "confidence": 3, "studyTime": 60}
        ]
    }
]

assignments_raw = [
    {
        "course": "Computer Networks", "title": "Midterm Essay", "associatedTopic": ["TCP/IP Model"],
        "dueDate": "14/02/2025", "time": "15.00", "estimatedTime": 50
    },
    {
        "course": "Software Architecture", "title": "Final Project", 
        "associatedTopic": ["Machine Learning", "Classification", "Neural Networks"], # Assuming this implies topics from ML course
        "dueDate": "17/02/2025", "estimatedTime": 180
    }
]

# --- Helper Functions (from previous steps, slightly adapted) ---
def round10(x):
    return math.ceil(x / 10) * 10

def estimate_study_time(difficulty, confidence, base_time=60):
    adjustment = (difficulty - confidence) * 0.1
    adjusted_time = base_time * (1 + adjustment)
    adjusted_time = max(15, adjusted_time)
    return round10(adjusted_time)

def get_session_types_for_freq(revision_freq):
    if revision_freq == "single deep review before exam":
        return ["deep", "review"]
    elif revision_freq == "2-3 reviews per topic":
        return ["overview", "deep", "review"]
    elif revision_freq == "daily review sessions": # Simplified for this context
        return ["daily_1", "daily_2", "review"] # Assume 2 daily + 1 review
    else:
        return ["core", "review"]


def estimate_topic_sessions_raw(adjusted_study_time, revision_freq, preferred_study_duration):
    min_dur, max_dur = preferred_study_duration["min"], preferred_study_duration["max"]
    sessions_temp = []
    if revision_freq == "single deep review before exam":
        deep = round10(adjusted_study_time)
        review = round10(adjusted_study_time * 0.2)
        sessions_temp = [deep, review]
    elif revision_freq == "2-3 reviews per topic":
        overview = round10(adjusted_study_time * 0.2)
        deep = round10(adjusted_study_time)
        review = round10(adjusted_study_time * 0.3)
        sessions_temp = [overview, deep, review]
    elif revision_freq == "daily review sessions":
        # Simplified: assume 3 sessions if daily, one main, two smaller reviews
        core_time = adjusted_study_time
        r_time = round10(core_time * 0.25)
        main_time = core_time - r_time # this is rough, needs better distribution
        sessions_temp = [round10(main_time/2), round10(main_time/2), r_time] if main_time > 0 else [r_time]

    else: # Default
        core = round10(adjusted_study_time)
        review = round10(adjusted_study_time * 0.2)
        sessions_temp = [core, review]
    
    final_sessions = []
    for s in sessions_temp:
        if s == 0: continue
        if s > max_dur:
            parts = math.ceil(s / max_dur)
            per_part = round10(s / parts)
            final_sessions.extend([per_part] * parts)
        elif s < min_dur:
             final_sessions.append(min_dur) # ensure min duration
        else:
            final_sessions.append(s)
    return [s for s in final_sessions if s > 0]


def create_assignment_sessions_raw(estimated_time, preferred_study_duration):
    min_dur, max_dur = preferred_study_duration["min"], preferred_study_duration["max"]
    assignment_sessions = []
    if estimated_time == 0: return []

    if estimated_time < min_dur :
        return [max(round10(estimated_time), min_dur)] # Ensure min_dur if any work

    if estimated_time > max_dur:
        parts = math.ceil(estimated_time / max_dur)
        per_part = round10(estimated_time / parts)
        temp_sessions = [per_part] * parts
        # Adjust last part if sum is off due to rounding parts up
        current_sum = sum(temp_sessions)
        if current_sum > estimated_time and len(temp_sessions) > 0: # try to reduce last session
            diff = current_sum - estimated_time
            temp_sessions[-1] = max(min_dur, temp_sessions[-1] - diff) # ensure not too small
            temp_sessions[-1] = round10(temp_sessions[-1])


        final_assignment_sessions = []
        for s_val in temp_sessions:
            if s_val < min_dur and s_val > 0 :
                final_assignment_sessions.append(min_dur)
            elif s_val > 0:
                final_assignment_sessions.append(s_val)
        return [s for s in final_assignment_sessions if s > 0] # clean zero/negatives
    else:
        return [max(round10(estimated_time),min_dur)]


def get_preferred_time_ranges(pref_strings):
    time_periods_map = {
        "early morning": (4, 8), "late morning": (8, 12), "afternoon": (12, 18),
        "evening": (18, 22), "night": (22, 24), "late night": (0, 4)
    }
    return [time_periods_map[p] for p in pref_strings if p in time_periods_map]

# --- Parsed Data Structures ---
all_schedulable_tasks = []
task_id_counter = 1

# 1. Process Courses and their Topics into Schedulable Tasks
for course in courses_raw:
    course_name = course["name"]
    exam_date_dt = datetime.strptime(course["examDate"], "%d/%m/%Y")
    
    # Topic sessions are prerequisites for each other in sequence
    previous_topic_session_id = None
    
    for topic_idx, topic in enumerate(course["topics"]):
        topic_title = topic["title"]
        adj_time = estimate_study_time(topic["difficulty"], topic["confidence"], topic["studyTime"])
        sessions_durations = estimate_topic_sessions_raw(adj_time, user_preferences["revisionFrequency"], user_preferences["preferredSessionDuration"])
        session_types = get_session_types_for_freq(user_preferences["revisionFrequency"])


        for i, duration in enumerate(sessions_durations):
            task_id = f"task_{task_id_counter}"
            task_id_counter += 1
            
            session_type_label = f"{session_types[i % len(session_types)]}_{i // len(session_types) + 1}" if len(sessions_durations) > len(session_types) else session_types[i]

            # Determine if it's a hard part (e.g., 'deep' or 'core' sessions)
            is_hard = "deep" in session_type_label or "core" in session_type_label or duration >= user_preferences["preferredSessionDuration"]["max"] * 0.8 # Heuristic for hard

            # Determine if it's the final review for this topic
            is_final = (i == len(sessions_durations) - 1)

            prereqs = [previous_topic_session_id] if previous_topic_session_id else []

            task = {
                "id": task_id,
                "type": "topic",
                "name": f"{course_name}: {topic_title} ({session_type_label})",
                "course_name": course_name,
                "topic_title": topic_title,
                "session_type": session_type_label,
                "duration": duration,
                "deadline": exam_date_dt, # Exam date is the ultimate deadline for topic study
                "prerequisites": prereqs,
                "is_hard_part": is_hard,
                "is_final_review_for_topic": is_final,
                "course_credit": course["credit"],
                "topic_difficulty": topic["difficulty"],
                "topic_confidence": topic["confidence"],
                "status": "pending",
                "scheduled_start": None,
                "scheduled_end": None,
                "associated_topics_for_assignment": None # For consistency
            }
            all_schedulable_tasks.append(task)
            previous_topic_session_id = task_id # Current session is prereq for next in same topic
        
        # After all sessions for a topic, the last session of THIS topic becomes a general prereq
        # for assignments that might depend on the whole topic being covered.
        # This simple model might need refinement for assignments depending on specific session types.
        # For now, an assignment depends on ALL sessions of its associated topics.

# 2. Process Assignments into Schedulable Tasks
for assignment in assignments_raw:
    assignment_title = assignment["title"]
    assignment_course = assignment["course"] # Could be different from courses_raw if it's a project course
    due_date_dt = datetime.strptime(assignment["dueDate"], "%d/%m/%Y")
    # Combine due_date with time if available, otherwise end of day
    if "time" in assignment:
        due_time_obj = datetime.strptime(assignment["time"], "%H.%M").time()
        due_date_dt = datetime.combine(due_date_dt.date(), due_time_obj)
    else: # Default to end of working day or specific time
        due_date_dt = datetime.combine(due_date_dt.date(), datetime.strptime("23:59", "%H:%M").time())


    sessions_durations = create_assignment_sessions_raw(assignment["estimatedTime"], user_preferences["preferredSessionDuration"])
    
    # Determine prerequisites: all topic sessions for associated topics
    assignment_prereqs = []
    for req_topic_title in assignment.get("associatedTopic", []):
        # Find all task IDs related to this topic from any course
        # (Assuming topic titles are unique enough or course context is implied)
        # A more robust system would qualify topic by course. For now, title match.
        for t in all_schedulable_tasks:
            if t["type"] == "topic" and t["topic_title"] == req_topic_title:
                 # An assignment should ideally depend on the *completion* of a topic,
                 # meaning all its sessions, particularly the final review.
                 if t["is_final_review_for_topic"]: # Or just all sessions of that topic
                    assignment_prereqs.append(t["id"])


    previous_assignment_session_id = None
    for i, duration in enumerate(sessions_durations):
        task_id = f"task_{task_id_counter}"
        task_id_counter += 1
        
        task = {
            "id": task_id,
            "type": "assignment",
            "name": f"Assignment: {assignment_title} (Part {i+1})",
            "course_name": assignment_course, # Store for context
            "topic_title": None, # Assignments don't have one primary topic session
            "session_type": f"work_part_{i+1}",
            "duration": duration,
            "deadline": due_date_dt,
            "prerequisites": list(set(assignment_prereqs + ([previous_assignment_session_id] if previous_assignment_session_id else []))), # All topic prereqs + previous part of same assignment
            "is_hard_part": duration >= user_preferences["preferredSessionDuration"]["max"] * 0.8, # Heuristic
            "is_final_review_for_topic": False, # N/A for assignments
            "course_credit": None, # Can be inferred if needed for prioritization
            "topic_difficulty": None,
            "topic_confidence": None,
            "status": "pending",
            "scheduled_start": None,
            "scheduled_end": None,
            "associated_topics_for_assignment": assignment.get("associatedTopic", [])
        }
        all_schedulable_tasks.append(task)
        previous_assignment_session_id = task_id


# 3. Process Availability
parsed_availability_slots = []
preferred_time_ranges_of_day = get_preferred_time_ranges(user_preferences["preferredStudyTimes"])

for date_str, time_slots_str in availability_raw.items():
    base_date = datetime.strptime(date_str, "%d/%m/%Y").date()
    for time_slot_str in time_slots_str:
        start_str, end_str = time_slot_str.split('-')
        start_dt = datetime.combine(base_date, datetime.strptime(start_str, "%H:%M").time())
        end_dt = datetime.combine(base_date, datetime.strptime(end_str, "%H:%M").time())
        
        duration_minutes = int((end_dt - start_dt).total_seconds() / 60)
        if duration_minutes <= 0: continue

        # Check if this slot overlaps with any preferred time period
        is_preferred = False
        slot_start_hour = start_dt.hour + start_dt.minute / 60.0
        slot_end_hour = end_dt.hour + end_dt.minute / 60.0
        if slot_end_hour == 0.0 and end_dt.minute == 0 : slot_end_hour = 24.0 # Handle midnight end

        for pref_start, pref_end in preferred_time_ranges_of_day:
            # Check for overlap: max(start1, start2) < min(end1, end2)
            overlap_start = max(slot_start_hour, pref_start)
            overlap_end = min(slot_end_hour, pref_end)
            if overlap_start < overlap_end:
                is_preferred = True
                break
        
        parsed_availability_slots.append({
            "id": f"slot_{date_str}_{start_str}",
            "start_datetime": start_dt,
            "end_datetime": end_dt,
            "total_duration_minutes": duration_minutes,
            "remaining_duration_minutes": duration_minutes,
            "is_preferred": is_preferred,
            "scheduled_tasks_in_slot": [] # To store (task_id, start, end)
        })

# Sort slots by start time
parsed_availability_slots.sort(key=lambda s: s["start_datetime"])

print(f"Total schedulable tasks: {len(all_schedulable_tasks)}")
for t in all_schedulable_tasks[:5]: print(t) # Print a few tasks
print(f"\nTotal availability slots: {len(parsed_availability_slots)}")
for s in parsed_availability_slots[:3]: print(s) # Print a few slots

Total schedulable tasks: 55
{'id': 'task_1', 'type': 'topic', 'name': 'Operating Systems: CPU Scheduling (overview_1)', 'course_name': 'Operating Systems', 'topic_title': 'CPU Scheduling', 'session_type': 'overview_1', 'duration': 30, 'deadline': datetime.datetime(2025, 2, 25, 0, 0), 'prerequisites': [], 'is_hard_part': False, 'is_final_review_for_topic': False, 'course_credit': 3, 'topic_difficulty': 3, 'topic_confidence': 2, 'status': 'pending', 'scheduled_start': None, 'scheduled_end': None, 'associated_topics_for_assignment': None}
{'id': 'task_2', 'type': 'topic', 'name': 'Operating Systems: CPU Scheduling (deep_1)', 'course_name': 'Operating Systems', 'topic_title': 'CPU Scheduling', 'session_type': 'deep_1', 'duration': 50, 'deadline': datetime.datetime(2025, 2, 25, 0, 0), 'prerequisites': ['task_1'], 'is_hard_part': True, 'is_final_review_for_topic': False, 'course_credit': 3, 'topic_difficulty': 3, 'topic_confidence': 2, 'status': 'pending', 'scheduled_start': None, 'scheduled

- bring the last session (review) before the exam date

In [181]:
import math
from datetime import datetime, timedelta, time
import copy

# --- DATA (Same as your previous input) ---
user_preferences = {
    "preferredStudyTimes": ["late morning", "afternoon", "night"],
    "preferredSessionDuration": { "min": 30, "max": 90 },
    "revisionFrequency": "2-3 reviews per topic",
    "breakDuration": 15
}

availability_raw = {
    "11/02/2025": ["10:00-12:00", "14:00-17:00", "20:00-22:00"],
    "12/02/2025": ["09:00-11:00", "14:00-16:00", "20:00-22:00"],
    "13/02/2025": ["10:00-12:00", "14:00-17:00", "20:00-22:00"],
    "14/02/2025": ["10:00-12:00", "14:00-18:00", "20:00-22:00"],
    "15/02/2025": ["08:00-11:00", "14:00-16:00", "20:00-22:00"],
    "17/02/2025": ["10:00-12:00", "14:00-17:00", "20:00-22:00"],
    "18/02/2025": ["10:00-12:00", "14:00-17:00", "20:00-22:00"],
    "19/02/2025": ["09:00-12:00", "14:00-17:00", "20:00-22:00"],
    "20/02/2025": ["09:00-12:00", "14:00-17:00", "20:00-22:00"],
    "21/02/2025": ["09:00-12:00", "14:00-17:00", "20:00-22:00"],
    "22/02/2025": ["09:00-12:00", "14:00-17:00", "20:00-22:00"],
    "24/02/2025": ["09:00-12:00", "14:00-17:00", "20:00-22:00"],
    "26/02/2025": ["09:00-12:00", "14:00-17:00", "20:00-22:00"],
}

courses_raw = [
    {
        "name": "Operating Systems", "credit": 3, "examDate": "25/02/2025", "examTime": "09:00–11:00",
        "topics": [
            {"title": "CPU Scheduling", "difficulty": 3, "confidence": 2, "studyTime": 90},
            {"title": "Deadlocks", "difficulty": 4, "confidence": 3, "studyTime": 75},
            {"title": "Virtual Memory", "difficulty": 5, "confidence": 2, "studyTime": 120}
        ]
    },
    {
        "name": "Data Structures and Algorithms", "credit": 3, "examDate": "22/02/2025", "examTime": "10:00–12:00",
        "topics": [
            {"title": "Trees & Graphs", "difficulty": 3, "confidence": 2, "studyTime": 80},
            {"title": "Sorting Algorithms", "difficulty": 2, "confidence": 4, "studyTime": 50},
            {"title": "Dynamic Programming", "difficulty": 5, "confidence": 2, "studyTime": 110}
        ]
    },
    {
        "name": "Machine Learning", "credit": 3, "examDate": "23/02/2025", "examTime": "13:00–15:00",
        "topics": [
            {"title": "Regression Models", "difficulty": 3, "confidence": 3, "studyTime": 70},
            {"title": "Classification", "difficulty": 4, "confidence": 2, "studyTime": 90},
            {"title": "Neural Networks", "difficulty": 5, "confidence": 2, "studyTime": 130}
        ]
    },
    {
        "name": "Database Systems", "credit": 2, "examDate": "27/02/2025", "examTime": "09:30–11:30",
        "topics": [
            {"title": "SQL Joins", "difficulty": 2, "confidence": 4, "studyTime": 40},
            {"title": "Normalization", "difficulty": 3, "confidence": 3, "studyTime": 60},
            {"title": "Indexing & Query Optimization", "difficulty": 4, "confidence": 2, "studyTime": 90}
        ]
    },
    {
        "name": "Computer Networks", "credit": 2, "examDate": "25/02/2025", "examTime": "14:00–16:00",
        "topics": [
            {"title": "TCP/IP Model", "difficulty": 2, "confidence": 4, "studyTime": 50},
            {"title": "Routing Protocols", "difficulty": 4, "confidence": 2, "studyTime": 100},
            {"title": "Congestion Control", "difficulty": 3, "confidence": 3, "studyTime": 60}
        ]
    }
]

assignments_raw = [
    {
        "course": "Computer Networks", "title": "Midterm Essay", "associatedTopic": ["TCP/IP Model"],
        "dueDate": "15/02/2025", "time": "15:00", "estimatedTime": 50
    },
    {
        "course": "Machine Learning", "title": "Final Project",
        "associatedTopic": ["Regression Models", "Classification", "Neural Networks"],
        "dueDate": "05/03/2025", "estimatedTime": 180
    }
]

# --- Timeline Reference ---
REFERENCE_DATE = datetime(2025, 2, 1, 0, 0, 0)

def to_minutes(dt):
    if dt is None: return None
    return int((dt - REFERENCE_DATE).total_seconds() / 60)

def from_minutes(minutes):
    if minutes is None: return None
    return REFERENCE_DATE + timedelta(minutes=minutes)

# --- Helper Functions ---
def round10(x):
    return math.ceil(x / 10.0) * 10

def estimate_study_time(difficulty, confidence, base_time=60):
    adjustment = (difficulty - confidence) * 0.1
    adjusted_time = base_time * (1 + adjustment)
    adjusted_time = max(15, adjusted_time)
    return round10(adjusted_time)

def estimate_topic_sessions_raw(adjusted_study_time, revision_freq, preferred_study_duration):
    min_dur, max_dur = preferred_study_duration["min"], preferred_study_duration["max"]
    sessions_temp = []
    if revision_freq == "single deep review before exam":
        sessions_temp = [int(round10(adjusted_study_time)), int(round10(adjusted_study_time * 0.2))]
    elif revision_freq == "2-3 reviews per topic":
        sessions_temp = [int(round10(adjusted_study_time * 0.2)), int(round10(adjusted_study_time)), int(round10(adjusted_study_time * 0.3))]
    else: # Default
        sessions_temp = [int(round10(adjusted_study_time)), int(round10(adjusted_study_time * 0.2))]

    final_sessions = []
    for s_duration in sessions_temp:
        if s_duration == 0: continue
        if s_duration > max_dur:
            parts = math.ceil(s_duration / float(max_dur))
            per_part = int(round10(s_duration / parts))
            final_sessions.extend([per_part] * int(parts))
        elif s_duration < min_dur :
            final_sessions.append(int(min_dur))
        else:
            final_sessions.append(int(s_duration))
    return [s for s in final_sessions if s > 0]

def create_assignment_sessions_raw(estimated_time, preferred_study_duration):
    min_dur, max_dur = preferred_study_duration["min"], preferred_study_duration["max"]
    if estimated_time == 0: return []
    if estimated_time < min_dur : return [int(max(round10(estimated_time), min_dur))]
    final_assignment_sessions = []
    if estimated_time > max_dur:
        parts = math.ceil(estimated_time / float(max_dur))
        per_part = int(round10(estimated_time / parts))
        final_assignment_sessions.extend([per_part] * int(parts))
    else:
        final_assignment_sessions.append(int(round10(estimated_time)))
    return [max(s, min_dur) for s in final_assignment_sessions if s > 0]

def get_time_period_type(dt_hour_float, preferred_time_names_list):
    time_periods_map = {
        "early morning": (4, 8), "late morning": (8, 12), "afternoon": (12, 18),
        "evening": (18, 22), "night": (22, 24), "late night": (0, 4)
    }
    # Check against all time periods first, then determine if it's a preferred one
    actual_period = "other" # Default
    for period_name, (start_h, end_h) in time_periods_map.items():
        if period_name == "late night" and start_h == 0:
             if dt_hour_float >= 0 and dt_hour_float < end_h:
                 actual_period = period_name
                 break
        elif dt_hour_float >= start_h and dt_hour_float < end_h:
            actual_period = period_name
            break
    # Return the actual period, the scheduler will decide if it's preferred based on user_prefs_dict
    return actual_period


# --- Data Preparation for Heuristic Scheduler ---
all_schedulable_tasks = []
task_id_counter = 0
topic_core_completion_task_ids = {}

for course in courses_raw:
    course_name = course["name"]
    exam_dt_str = course["examDate"] + " " + course["examTime"].split('–')[0]
    exam_deadline_dt = datetime.strptime(exam_dt_str, "%d/%m/%Y %H:%M")
    for topic in course["topics"]:
        topic_title = topic["title"]
        adj_time = estimate_study_time(topic["difficulty"], topic["confidence"], topic["studyTime"])
        sessions_durations = estimate_topic_sessions_raw(adj_time, user_preferences["revisionFrequency"], user_preferences["preferredSessionDuration"])
        previous_session_id_in_topic = None
        last_core_session_id_for_this_topic_in_loop = None

        for i, duration in enumerate(sessions_durations):
            task_id = task_id_counter
            task_id_counter += 1
            session_label = f"S{i+1}"
            prereqs = []
            if previous_session_id_in_topic is not None: prereqs.append(previous_session_id_in_topic)
            task_display_name = f"{course_name} - {topic_title} - {session_label}"
            is_this_final_review = (i == len(sessions_durations) - 1)
            task = {
                "id": task_id, "type": "topic", "name": task_display_name,
                "duration_minutes": duration, "deadline_datetime": exam_deadline_dt,
                "prerequisites": prereqs, "is_final_review": is_this_final_review,
                "course_name": course_name, "topic_title": topic_title,
                "exam_datetime": exam_deadline_dt,
                "original_difficulty": topic["difficulty"], "original_confidence": topic["confidence"],
                "course_credit": course["credit"], "session_type_raw": "topic_session",
                "status": "pending", "scheduled_start_abs_min": None, "scheduled_end_abs_min": None
            }
            all_schedulable_tasks.append(task)
            previous_session_id_in_topic = task_id
            if not is_this_final_review:
                last_core_session_id_for_this_topic_in_loop = task_id
            elif is_this_final_review and last_core_session_id_for_this_topic_in_loop is None : # Handles topics with only one session that is also the final review
                last_core_session_id_for_this_topic_in_loop = task_id


        if last_core_session_id_for_this_topic_in_loop is not None:
            topic_key = (course_name, topic_title)
            topic_core_completion_task_ids[topic_key] = last_core_session_id_for_this_topic_in_loop
        elif previous_session_id_in_topic is not None: # Fallback if only one session and it wasn't caught by above
             topic_key = (course_name, topic_title)
             topic_core_completion_task_ids[topic_key] = previous_session_id_in_topic


for assignment in assignments_raw:
    assignment_title = assignment["title"]
    course_name_for_assignment = assignment.get("course", "General Assignment")
    due_date_dt_str = assignment["dueDate"]
    due_date_dt_str += f" {assignment.get('time', '23:59').replace('.',':')}" # Ensure time is parsed correctly
    due_date_dt = datetime.strptime(due_date_dt_str, "%d/%m/%Y %H:%M")
    sessions_durations = create_assignment_sessions_raw(assignment["estimatedTime"], user_preferences["preferredSessionDuration"])
    assignment_prereqs = []
    for req_topic_title in assignment.get("associatedTopic", []):
        found_prereq = False
        for (c_name, t_title), task_id_val in topic_core_completion_task_ids.items():
            if t_title == req_topic_title and \
               (c_name == course_name_for_assignment or \
                (course_name_for_assignment == "Software Architecture" and c_name == "Machine Learning") or \
                 assignment.get("course") == c_name): # Ensure course name matches if specified
                assignment_prereqs.append(task_id_val)
                found_prereq = True
                break
        # if not found_prereq:
            # print(f"Warning: Core prerequisite topic '{req_topic_title}' for assignment '{assignment_title}' not definitively found.")

    previous_assignment_session_id = None
    for i, duration in enumerate(sessions_durations):
        task_id = task_id_counter
        task_id_counter += 1
        session_label = f"Part {i+1}"
        prereqs_for_part = list(set(assignment_prereqs)) # Ensure unique prerequisites from topics
        if previous_assignment_session_id is not None: prereqs_for_part.append(previous_assignment_session_id)
        task_display_name = f"{course_name_for_assignment} - {assignment_title} - {session_label}"
        task = {
            "id": task_id, "type": "assignment", "name": task_display_name,
            "duration_minutes": duration, "deadline_datetime": due_date_dt,
            "prerequisites": prereqs_for_part, "is_final_review": False, # Assignments are not 'final reviews'
            "course_name": course_name_for_assignment, "topic_title": None, # Assignments don't have a single topic title
            "original_difficulty": None, "original_confidence": None, "course_credit": None, # Not applicable
            "session_type_raw": "assignment_part",
            "status": "pending", "scheduled_start_abs_min": None, "scheduled_end_abs_min": None
        }
        all_schedulable_tasks.append(task)
        previous_assignment_session_id = task_id

parsed_availability_slots_abs = []
for date_str, time_slots_str in availability_raw.items():
    base_date = datetime.strptime(date_str, "%d/%m/%Y").date()
    for time_slot_str in time_slots_str:
        start_str, end_str = time_slot_str.split('-')
        start_dt = datetime.combine(base_date, datetime.strptime(start_str, "%H:%M").time())
        end_dt = datetime.combine(base_date, datetime.strptime(end_str, "%H:%M").time())
        slot_start_min = to_minutes(start_dt)
        slot_end_min = to_minutes(end_dt)
        if slot_end_min > slot_start_min: # Ensure slot is valid
            slot_start_hour_float = start_dt.hour + start_dt.minute / 60.0
            # Pass the full list of preferred times to get_time_period_type
            # The function itself will return the period type (e.g. "late morning")
            # The scheduler will then check if this type is in user_preferences["preferredStudyTimes"]
            pref_type = get_time_period_type(slot_start_hour_float, user_preferences["preferredStudyTimes"])
            parsed_availability_slots_abs.append({
                "start_abs_min": slot_start_min, "end_abs_min": slot_end_min,
                "duration": slot_end_min - slot_start_min, "preference_type": pref_type
            })
parsed_availability_slots_abs.sort(key=lambda x: x['start_abs_min'])


# --- Time Reduction Logic ---
def reduce_task_durations_if_needed(tasks_list, availability_slots, min_session_duration_pref, reduction_step=10):
    tasks = copy.deepcopy(tasks_list)
    total_available_time = sum(slot['duration'] for slot in availability_slots)
    while True:
        current_total_required_time = sum(task['duration_minutes'] for task in tasks)
        if current_total_required_time <= total_available_time:
            print(f"\nTime Reduction: Total required time ({current_total_required_time} min) is now within available time ({total_available_time} min).")
            break
        time_to_cut = current_total_required_time - total_available_time
        print(f"\nTime Reduction: Need to cut {time_to_cut} more minutes. Required: {current_total_required_time}, Available: {total_available_time}")
        best_task_to_reduce = None
        highest_reducibility_score = -float('inf') # Ensure any valid score is higher
        for task_idx, task in enumerate(tasks):
            if task['duration_minutes'] > min_session_duration_pref:
                score = 0
                # Score calculation logic (same as before)
                if task['type'] == 'topic':
                    score += (task.get('original_confidence', 0) or 0) * 3
                    score -= (task.get('original_difficulty', 5) or 5) * 2
                    score -= (task.get('course_credit', 3) or 3) * 1
                    session_num_char = task['name'][-1] if task['name'][-2] == 'S' else None
                    if task.get('is_final_review', False) or session_num_char in ['3', '4']: score += 5 # Final reviews and later sessions are more reducible
                    elif session_num_char == '1': score += 3 # First sessions are less reducible
                elif task['type'] == 'assignment':
                    session_num_char = task['name'][-1] if 'Part ' in task['name'] and task['name'][-1].isdigit() else None
                    if session_num_char and int(session_num_char) > 1 : score -= 2 # Later parts slightly more reducible
                    else: score -= 5 # First parts of assignments are less reducible

                if score > highest_reducibility_score:
                    highest_reducibility_score = score
                    best_task_to_reduce = task

        if best_task_to_reduce:
            original_duration = best_task_to_reduce['duration_minutes']
            new_duration = max(min_session_duration_pref, original_duration - reduction_step)
            if new_duration < original_duration: # Ensure reduction actually happens
                print(f"  Reducing task '{best_task_to_reduce['name']}' (ID: {best_task_to_reduce['id']}) from {original_duration} to {new_duration} min. Score: {highest_reducibility_score:.2f}")
                best_task_to_reduce['duration_minutes'] = new_duration
            else: # Cannot reduce further or no actual change
                print("  No more tasks can be meaningfully reduced (already at min or reduction step too small). Stopping reduction.")
                break
        else:
            print("  No suitable tasks found for further reduction. Stopping reduction.")
            break
    final_required_time = sum(task['duration_minutes'] for task in tasks)
    if final_required_time > total_available_time:
        print(f"Warning: After reduction, required time ({final_required_time} min) still exceeds available ({total_available_time} min).")
    return tasks

# --- Heuristic Scheduler with Preference Scoring & Effective Deadlines ---
def heuristic_scheduler(tasks_to_schedule, raw_availability_slots_with_prefs, user_prefs_dict):
    tasks = copy.deepcopy(tasks_to_schedule)
    tasks_by_id = {task['id']: task for task in tasks}

    # Propagate effective deadlines
    for task in tasks:
        task['effective_deadline'] = task['deadline_datetime'] # Initialize with actual deadline

    for _ in range(len(tasks)): # Iterate to ensure propagation
        changed_in_pass = False
        for task_data_val in tasks: # Iterate over a copy or be careful with modification
            for prereq_id_val in task_data_val['prerequisites']:
                if prereq_id_val in tasks_by_id: # Check if prereq_id is valid
                    prereq_task = tasks_by_id[prereq_id_val]
                    if task_data_val['effective_deadline'] < prereq_task['effective_deadline']:
                        prereq_task['effective_deadline'] = task_data_val['effective_deadline']
                        changed_in_pass = True
        if not changed_in_pass:
            break
            
    tasks.sort(key=lambda t: (t['effective_deadline'], t['deadline_datetime'], t['id']))

    dynamic_availability_windows = sorted(
        [{'start': s['start_abs_min'], 'end': s['end_abs_min'], 
          'preference_type': s['preference_type']} for s in raw_availability_slots_with_prefs],
        key=lambda x: x['start']
    )

    for task_idx, task in enumerate(tasks):
        if task['status'] == 'scheduled': continue

        prereqs_met = True
        latest_prereq_end_time = 0
        for prereq_id in task['prerequisites']:
            prereq_task = tasks_by_id.get(prereq_id) # Use .get for safety
            if not prereq_task or prereq_task['status'] != 'scheduled':
                prereqs_met = False
                break
            if prereq_task['scheduled_end_abs_min'] is not None: # Ensure it was scheduled
                latest_prereq_end_time = max(latest_prereq_end_time, prereq_task['scheduled_end_abs_min'])
        
        if not prereqs_met: continue

        task_duration = task['duration_minutes']
        
        candidate_placements = []
        # Iterate over a copy of dynamic_availability_windows if modifying it inside the loop by index
        # or be careful with how windows are removed/added
        for window_idx, window in enumerate(list(dynamic_availability_windows)): # Iterate over a copy for safety if modifying list
            potential_start = max(window['start'], latest_prereq_end_time)
            potential_end = potential_start + task_duration

            if potential_start >= window['end'] or potential_end > window['end']: continue
            if potential_end > to_minutes(task['deadline_datetime']): continue # Check against actual deadline
            
            # Special handling for final reviews (same as before)
            if task.get('is_final_review', False) and task.get('exam_datetime'):
                exam_dt = task['exam_datetime']
                min_review_start_dt = datetime.combine(exam_dt.date() - timedelta(days=2), time.min)
                min_review_start_abs = to_minutes(min_review_start_dt)
                if potential_start < min_review_start_abs : 
                    continue 
            
            score = 0
            score -= potential_start # Prioritize earlier slots generally

            # MODIFICATION: Apply time-of-day preference scoring ONLY for 'topic' tasks
            if task['type'] == 'topic':
                window_pref_type = window['preference_type'] # This is the actual period type of the window
                if window_pref_type in user_prefs_dict["preferredStudyTimes"]:
                    if window_pref_type == "night": score += 50000
                    elif window_pref_type == "afternoon": score += 20000
                    elif window_pref_type == "late morning": score += 10000
                elif window_pref_type == "other": # Penalize 'other' times for topics
                    score -= 10000
            # For 'assignment' tasks, no additional score adjustment based on time-of-day preference.

            candidate_placements.append({
                "window_original_ref": window, # Keep reference to original window object
                "start": potential_start, 
                "end": potential_end, 
                "score": score
            })

        if candidate_placements:
            candidate_placements.sort(key=lambda p: p['score'], reverse=True)
            best_placement = candidate_placements[0]
            task['status'] = 'scheduled'
            task['scheduled_start_abs_min'] = best_placement['start']
            task['scheduled_end_abs_min'] = best_placement['end']
            
            window_to_modify = best_placement['window_original_ref'] 
            
            # Window modification logic (robustly find and update)
            # This needs to be careful if window_to_modify might have been removed/split by a previous task
            # For simplicity, assuming window_to_modify is a direct reference that can be removed
            # More robust: find by exact start/end or use indices if not iterating over a copy.
            try:
                current_window_index = -1
                for i, w in enumerate(dynamic_availability_windows):
                    if w['start'] == window_to_modify['start'] and w['end'] == window_to_modify['end'] and w['preference_type'] == window_to_modify['preference_type']:
                        current_window_index = i
                        break
                
                if current_window_index != -1:
                    dynamic_availability_windows.pop(current_window_index) # Remove the chosen window
                    new_windows_to_add = []
                    if best_placement['start'] > window_to_modify['start']:
                        new_windows_to_add.append({'start': window_to_modify['start'], 
                                                   'end': best_placement['start'], 
                                                   'preference_type': window_to_modify['preference_type']})
                    if best_placement['end'] < window_to_modify['end']:
                        new_windows_to_add.append({'start': best_placement['end'], 
                                                   'end': window_to_modify['end'], 
                                                   'preference_type': window_to_modify['preference_type']})
                    dynamic_availability_windows.extend(new_windows_to_add)
                    dynamic_availability_windows.sort(key=lambda x: x['start']) # Re-sort after adding new windows
                else:
                    # This indicates the window was already modified or removed, potentially by a task segmenting it.
                    # This can happen if a large window was partially consumed.
                    # A more complex window management would be needed if this becomes a frequent issue.
                    # For now, we assume this means the specific sub-segment this task fits into was found.
                    # The critical part is that the availability is correctly reduced.
                    # If this 'else' block is hit, it means our simple `remove(window_to_modify)` approach
                    # was insufficient due to prior modifications. The current code tries to find by value.
                    # print(f"Warning: Window {window_to_modify} not found directly, might have been split. Manual check needed if issues.")
                    # One strategy if a window is "split" is to find the window that *contains* the best_placement
                    # and then split that one.
                    pass


            except ValueError:
                # This case implies window_to_modify was not found, which could happen if it was split by a previous task.
                # The logic should ideally find the encompassing window and split it.
                # For now, we'll print a warning if this rare case happens.
                # print(f"Warning: Could not remove window {window_to_modify}, it might have been split already.")
                pass # Fallback, hoping the window was consumed correctly

        else: # No candidate placements found
            task['status'] = 'unscheduled'
            # print(f"Could not schedule: {task['name']}")
    return tasks

# --- Output Formatting ---
def print_formatted_schedule(scheduled_tasks_list):
    if not scheduled_tasks_list:
        print("No tasks were scheduled.")
        return

    print("\n🗓️ Study Plan:")
    tasks_to_print = sorted(
        [t for t in scheduled_tasks_list if t['status'] == 'scheduled' and t['scheduled_start_abs_min'] is not None],
        key=lambda t: t['scheduled_start_abs_min']
    )
    current_date_str = None
    for task in tasks_to_print:
        start_dt = from_minutes(task['scheduled_start_abs_min'])
        end_dt = from_minutes(task['scheduled_end_abs_min'])
        date_str = start_dt.strftime("%d/%m/%Y")
        if date_str != current_date_str:
            print(f"\n{date_str}") # Add a newline for better date separation
            current_date_str = date_str
        time_str = f"{start_dt.strftime('%H:%M')} - {end_dt.strftime('%H:%M')}"
        print(f"  {time_str} ➔ {task['name']}")

    unscheduled_tasks = [t for t in scheduled_tasks_list if t['status'] != 'scheduled']
    if unscheduled_tasks:
        print("\n❌ Unscheduled Tasks:")
        # Sort unscheduled tasks by effective deadline, then original deadline for clarity
        unscheduled_tasks.sort(key=lambda t: (t.get('effective_deadline', t['deadline_datetime']), t['deadline_datetime']))
        for task in unscheduled_tasks:
            print(f"  • {task['name']} (Duration: {task['duration_minutes']} min, Deadline: {task['deadline_datetime'].strftime('%Y-%m-%d %H:%M')}, Effective: {task.get('effective_deadline', task['deadline_datetime']).strftime('%Y-%m-%d %H:%M')}, Prereqs: {task['prerequisites']})")

# --- Main Execution ---
if __name__ == "__main__":
    if not all_schedulable_tasks:
        print("No tasks were generated. Exiting.")
    elif not parsed_availability_slots_abs:
        print("No availability slots parsed. Exiting.")
    else:
        print(f"\nInitially generated {len(all_schedulable_tasks)} tasks.")
        initial_total_required_time = sum(task['duration_minutes'] for task in all_schedulable_tasks)
        total_available_time_in_slots = sum(slot['duration'] for slot in parsed_availability_slots_abs)
        print(f"Initial total required time: {initial_total_required_time} min.")
        print(f"Total available time in slots: {total_available_time_in_slots} min.")

        tasks_for_scheduler = all_schedulable_tasks
        if initial_total_required_time > total_available_time_in_slots:
            print("Attempting to reduce task durations as required time exceeds available time...")
            tasks_for_scheduler = reduce_task_durations_if_needed(
                all_schedulable_tasks, 
                parsed_availability_slots_abs,
                user_preferences["preferredSessionDuration"]["min"]
            )
            final_required_time_after_reduction = sum(task['duration_minutes'] for task in tasks_for_scheduler)
            print(f"Total required time after reduction attempts: {final_required_time_after_reduction} min.")
        else:
            print("No time reduction needed as initial required time is within available time.")
        
        print(f"\nRunning Heuristic Scheduler for {len(tasks_for_scheduler)} tasks (with potentially adjusted durations)...")
        final_task_list_with_schedule = heuristic_scheduler(tasks_for_scheduler, parsed_availability_slots_abs, user_preferences)
        
        print_formatted_schedule(final_task_list_with_schedule)


Initially generated 55 tasks.
Initial total required time: 2700 min.
Total available time in slots: 5820 min.
No time reduction needed as initial required time is within available time.

Running Heuristic Scheduler for 55 tasks (with potentially adjusted durations)...

🗓️ Study Plan:

11/02/2025
  10:00 - 10:40 ➔ Operating Systems - Virtual Memory - S1
  10:40 - 12:00 ➔ Operating Systems - Virtual Memory - S2
  14:00 - 14:30 ➔ Computer Networks - TCP/IP Model - S1
  14:30 - 15:10 ➔ Computer Networks - TCP/IP Model - S2
  15:10 - 15:40 ➔ Data Structures and Algorithms - Trees & Graphs - S1
  15:40 - 16:10 ➔ Data Structures and Algorithms - Sorting Algorithms - S1
  16:10 - 16:50 ➔ Data Structures and Algorithms - Sorting Algorithms - S2

12/02/2025
  09:00 - 09:30 ➔ Computer Networks - Routing Protocols - S1
  09:30 - 10:30 ➔ Computer Networks - Routing Protocols - S2
  10:30 - 11:00 ➔ Computer Networks - Congestion Control - S1
  14:00 - 15:30 ➔ Data Structures and Algorithms - Trees 

In [182]:
for course in courses:
    print(f"\n📘 Course: {course['name']}")
    for topic in course["topics"]:
        base_time = topic.get("studyTime", 60)
        adjusted_time = estimate_study_time(topic["difficulty"], topic["confidence"], base_time)
        sessions = estimate_topic_sessions_raw(adjusted_time, user_preferences["revisionFrequency"], user_preferences["preferredSessionDuration"])
        session_info = ", ".join([f"{int(s)} min" for s in sessions])
        print(f"  • {topic['title']} → Total: {adjusted_time} min → Sessions: {session_info}")


📘 Course: Operating Systems
  • CPU Scheduling → Total: 100 min → Sessions: 30 min, 50 min, 50 min, 30 min
  • Deadlocks → Total: 90 min → Sessions: 30 min, 90 min, 30 min
  • Virtual Memory → Total: 160 min → Sessions: 40 min, 80 min, 80 min, 50 min

📘 Course: Data Structures and Algorithms
  • Trees & Graphs → Total: 90 min → Sessions: 30 min, 90 min, 30 min
  • Sorting Algorithms → Total: 40 min → Sessions: 30 min, 40 min, 30 min
  • Dynamic Programming → Total: 150 min → Sessions: 30 min, 80 min, 80 min, 50 min

📘 Course: Machine Learning
  • Regression Models → Total: 70 min → Sessions: 30 min, 70 min, 30 min
  • Classification → Total: 110 min → Sessions: 30 min, 60 min, 60 min, 40 min
  • Neural Networks → Total: 170 min → Sessions: 40 min, 90 min, 90 min, 60 min

📘 Course: Database Systems
  • SQL Joins → Total: 40 min → Sessions: 30 min, 40 min, 30 min
  • Normalization → Total: 60 min → Sessions: 30 min, 60 min, 30 min
  • Indexing & Query Optimization → Total: 110 min → Ses